# Jupyter Notebook for Diffusion-Based Generation

## Install Necessary Libraries

In this section, we install the required libraries for our project:
- `music2latent` for using the **Music2Latent** neural audio codec.
- `soundfile` for loading and saving audio files.

In [1]:
# Install necessary libraries (run this cell first)
!pip install music2latent soundfile

## Import Libraries

In this section, we import all the necessary libraries required for our project. This includes PyTorch for deep learning, `soundfile` for audio processing, `tqdm` for progress bars, and the `music2latent` package for encoding and decoding audio into latent representations.

In [3]:
import os
import torch
import scipy.signal
import torch.nn as nn
import soundfile as sf
import IPython.display as ipd
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from music2latent import EncoderDecoder

## Set Device

Here, we set the device for PyTorch computations. If a GPU is available, we use it for faster computations; otherwise, we fall back to the CPU.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Download NSYNTH_GUITAR_MP3 Dataset

In this section, we download the guitar subset of the NSynth dataset. We use `git-lfs` to handle large files and download the compressed dataset.

In [6]:
!git clone https://github.com/SonyCSLParis/test-lfs.git
!bash ./test-lfs/download.sh NSYNTH_GUITAR_MP3

Cloning into 'test-lfs'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 42 (delta 5), reused 40 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (42/42), 5.92 KiB | 466.00 KiB/s, done.
--2024-10-21 16:31:02--  https://media.githubusercontent.com/media/SonyCSLParis/test-lfs/refs/heads/master/NSYNTH_GUITAR_MP3.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 334999208 (319M) [application/zip]
Saving to: ‘NSYNTH_GUITAR_MP3.zip’

NSYNTH_GUITAR_MP3.z 100%[===================>] 319.48M  49.3MB/s    in 5.4s    

2024-10-21 16:31:24 (58.8 MB/s) - ‘NSYNTH_GUITAR_MP3.zip’ saved [334999208/334999208]

Fix archive (-F) - assume mostly intact archive


  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_acoustic_004-071-025.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_electronic_036-102-075.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_electronic_005-030-050.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_synthetic_006-042-025.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_electronic_037-043-127.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_synthetic_008-047-075.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_electronic_032-096-127.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_electronic_034-013-025.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_acoustic_008-053-127.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_electronic_003-071-100.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train/guitar_acoustic_008-023-100.mp3  
  inflating: NSYNTH_GUITAR_MP3/nsynth-guitar-train

## Define the Dataset Class

We define a custom `Dataset` class called `MusicLatentDataset`. This class handles the loading of audio files, encoding them into latent representations using the `music2latent` encoder, and standardizing the latent data. This prepares the data for training our diffusion model.

In [7]:
# Define the dataset class
class MusicLatentDataset(Dataset):
    def __init__(self, root_dir, encoder, extensions=[".wav", ".mp3", ".flac"], max_samples=-1):
        self.root_dir = root_dir
        self.encoder = encoder
        self.extensions = extensions
        self.audio_files = []

        # Walk through all subfolders to gather audio files
        for root, _, files in os.walk(root_dir):
            for file in files:
                if any(file.endswith(ext) for ext in self.extensions):
                    self.audio_files.append(os.path.join(root, file))

        # Limit the number of samples if max_samples is set
        if max_samples > 0:
            self.audio_files = self.audio_files[:max_samples]

        # Encode all audio files into latent representations
        self.latent_data = [self._encode_audio(file) for file in tqdm(self.audio_files, desc="Encoding Audio")]

        # Stack latent data and compute mean and standard deviation for normalization
        latent_data = torch.hstack(self.latent_data)
        self.mean = latent_data.mean(dim=1)
        self.std = latent_data.std(dim=1)

        # Standardize the latent data
        self.latent_data = [((latent.permute(1, 0) - self.mean) / self.std).permute(1, 0) for latent in self.latent_data]

        # Save mean and std for later use
        torch.save({'mean': self.mean, 'std': self.std}, 'mean_std.pth')

    def __len__(self):
        return len(self.latent_data)

    def __getitem__(self, idx):
        latent = self.latent_data[idx]
        return latent.float()

    def _encode_audio(self, filename):
        # Load the audio file using soundfile
        waveform, sample_rate = sf.read(filename)

        # Ensure it's in float32 precision
        waveform = waveform.astype('float32')

        # Resample to 44100 Hz if necessary
        if sample_rate != 44100:
            num_samples = int(len(waveform) * 44100 / sample_rate)
            waveform = scipy.signal.resample(waveform, num_samples)
            sample_rate = 44100

        # Encode using music2latent
        latent = self.encoder.encode(waveform)

        # Ensure latent is in float32
        latent = latent.float()

        # Remove batch dimension if necessary and limit to first 32 timesteps
        latent = latent.squeeze(0)

        return latent[..., :32]

    def unnormalize(self, latent):
        # Unnormalize the latent data using stored mean and std
        return latent * self.std + self.mean

## Define the Diffusion U-Net Model

Here, we define the `DiffusionUnet` class, which is a U-Net architecture adapted for diffusion modeling. The model includes time conditioning to allow the network to adjust its predictions based on the diffusion time step.

In [8]:
class DiffusionUnet(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        num_layers=3,
        base_channels=128,
        time_embedding_size=128,
    ):
        super(DiffusionUnet, self).__init__()
        self.num_layers = num_layers

        # Store channels at each layer to match encoder and decoder
        self.channels = []

        # Downsampling path (encoder)
        self.down_layers = nn.ModuleList()
        in_channels_current = in_channels
        for i in range(num_layers):
            out_channels_current = base_channels * (2 ** i)
            conv = nn.Conv1d(
                in_channels_current,
                out_channels_current,
                kernel_size=4,
                stride=2,
                padding=1,
            )
            self.down_layers.append(conv)
            self.channels.append(out_channels_current)
            in_channels_current = out_channels_current

        # Bottleneck layer
        self.bottleneck = nn.Conv1d(
            in_channels_current, in_channels_current, kernel_size=3, padding=1
        )

        # Time embedding to condition on diffusion time step
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_embedding_size),
            nn.ReLU(),
            nn.Linear(time_embedding_size, in_channels_current),
            nn.ReLU(),
        )

        # Upsampling path (decoder)
        self.up_layers = nn.ModuleList()
        for i in reversed(range(num_layers)):
            out_channels_current = in_channels_current // 2
            upsample = nn.ConvTranspose1d(
                in_channels_current,
                out_channels_current,
                kernel_size=4,
                stride=2,
                padding=1,
            )
            self.up_layers.append(upsample)
            in_channels_current = out_channels_current  # Update for next layer

        # Final output layer
        self.final_layer = nn.Conv1d(
            in_channels_current, out_channels, kernel_size=1
        )

    def forward(self, x, t):
        # Ensure input is float32
        x = x.float()
        t = t.float()

        # Downsampling path
        skip_connections = []
        h = x
        for down in self.down_layers:
            h = F.relu(down(h))
            skip_connections.append(h)

        # Bottleneck
        h = self.bottleneck(h)

        # Time embedding
        t_emb = self.time_mlp(t.unsqueeze(-1))  # [batch_size, channels]
        t_emb = t_emb.unsqueeze(-1)             # [batch_size, channels, 1]
        h = h + t_emb                           # Add time embedding

        # Upsampling path
        for up in self.up_layers:
            skip = skip_connections.pop()
            h = h + skip  # Skip connection
            h = F.relu(up(h))

        # Final output layer
        out = self.final_layer(h)

        return out

## Define the RectifiedFlows Class

The `RectifiedFlows` class implements the Rectified Flow method for diffusion modeling. It includes functions for adding noise to the data and calculating the training loss. This approach is based on learning a vector field that points towards regions of high data density.

In [9]:
# Define the RectifiedFlows class
class RectifiedFlows(torch.nn.Module):
    def __init__(self,
                 sigma_data=1,
                 # Expected mean and standard deviation of the training data.
                 P_mean=0.,
                 P_std=1.
                 ):
        super().__init__()
        self.sigma_data = sigma_data
        self.P_std = P_std
        self.P_mean = P_mean

    def add_noise(self, x, noise, times):
        # Linearly interpolate between data and noise based on times
        if isinstance(times, int):
            times = float(times)
        if isinstance(times, float):
            times = torch.ones((x.shape[0],), dtype=x.dtype, device=x.device) * times
        if len(times.shape) == 1:
            shape = [times.shape[0]] + (x.ndim - 1)*[1]
            times = times.reshape(shape)
        elif len(times.shape) == 2:
            shape = [times.shape[0]]+ [1]*(x.ndim - 2) + [-1]
            times = times.reshape(shape)
        return (1. - times) * x + times * noise

    def forward(self, model, x, sigma=None, return_loss=True,
                **model_kwargs) -> torch.Tensor:
        # Scale the data
        x = x.float() * self.sigma_data
        # Sample random times using a sigmoid of normal distribution
        times = torch.nn.functional.sigmoid(
            torch.randn(x.shape[0], dtype=torch.float32,
                        device=x.device) * self.P_std + self.P_mean
        )
        # Sample random noise
        noises = torch.randn_like(x)
        # Calculate the residual (difference between data and noise)
        v = x - noises
        # Add noise to the data
        noisy_samples = self.add_noise(x, noises, times)
        # Model prediction
        fv = model(
            noisy_samples,
            times,
            **model_kwargs
        )
        # Calculate the loss
        mse = nn.MSELoss()
        loss = mse(v, fv)
        if return_loss:
            return {'loss': loss}, (fv + noises) / self.sigma_data

## Define the Inference Function

The `inference` function performs ODE-based sampling to generate new samples from the trained diffusion model. It integrates over time from pure noise ($t=1$) to clean data ($t=0$), updating the sample at each step based on the model's predictions.

In [10]:
def inference(rectified_flows, net, latents_shape, num_steps):
    sigma_data = rectified_flows.sigma_data
    dtype = torch.float32  # Ensuring float32
    # Initialize with pure noise
    current_sample = torch.randn(latents_shape, dtype=dtype, device=next(net.parameters()).device)
    times = torch.ones(latents_shape[0], dtype=dtype, device=current_sample.device)
    # Calculate step size
    step_size = 1 / num_steps
    for i in tqdm(range(num_steps), desc="Sampling", leave=False):
        # Predict the residual
        v = net(
            current_sample,
            times
        )
        # Update the sample
        current_sample = current_sample + step_size * v
        # Decrease time
        times = times - step_size
    return current_sample / sigma_data

## Initialize the Encoder/Decoder and Datasets

In this section, we initialize the `EncoderDecoder` from `music2latent` and create instances of our `MusicLatentDataset` for both training and validation sets. The dataset class encodes all audio files into latent representations and standardizes them.

In [12]:
# Initialize the encoder/decoder
encdec = EncoderDecoder()

# Initialize the dataset and dataloader
audio_folder_train = "./NSYNTH_GUITAR_MP3/nsynth-guitar-train"
audio_folder_val = "./NSYNTH_GUITAR_MP3/nsynth-guitar-valid"

# When initializing the dataset, the data gets compressed using the M2L encoder
dataset = MusicLatentDataset(root_dir=audio_folder_train, encoder=encdec, max_samples=-1)
dataset_val = MusicLatentDataset(root_dir=audio_folder_val, encoder=encdec, max_samples=-1)


Encoding Audio: 100%|██████████| 2081/2081 [00:34<00:00, 61.19it/s]


In [13]:
# Create data loaders
dataloader = DataLoader(dataset, batch_size=500, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=500, shuffle=False)

## Initialize the Model

Here, we initialize an instance of our `DiffusionUnet` model, specifying the input and output channels, and the number of layers. The model is moved to the specified device (CPU or GPU).

In [14]:
# Initialize the model
in_channels = 64  # latent dimension
out_channels = 64  # output dimension
model = DiffusionUnet(in_channels, out_channels, num_layers=5).to(device)

## (Optional) Load Pretrained Weights

If available, you can load pretrained weights for the U-Net model to continue training or for inference.

In [ ]:
# Load pretrained model if available
# model.load_state_dict(torch.load('model_diffusion.pth'))

## Initialize RectifiedFlows

We create an instance of the `RectifiedFlows` class, which will be used during training to handle noise addition and loss calculation.

In [15]:
# Initialize RectifiedFlows
rectified_flows = RectifiedFlows().to(device)

## Training and Validation Loop

In this section, we define the training and validation loops. During training, we use the `RectifiedFlows` class to add noise to the data and compute the loss based on the model's predictions. We also perform validation to monitor the model's performance on unseen data.

In [23]:
from tqdm import tqdm

epochs = 1000
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
avg_loss = -1
avg_val_loss = -1

# Progress bar to track epoch-level progress
progress_bar = tqdm(range(epochs), desc=f"Epoch 1/{epochs}, Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

for epoch in progress_bar:
    model.train()
    total_loss = 0
    for batch in dataloader:
        x = batch.to(device).float()  # Ensure x is float32
        # Forward pass
        loss_dict, _ = rectified_flows(model, x)
        loss = loss_dict['loss']
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)

    # Validation loop
    model.eval()
    val_total_loss = 0
    with torch.no_grad():
        for batch in dataloader_val:
            x = batch.to(device).float()
            # Forward pass
            loss_dict, _ = rectified_flows(model, x)
            loss = loss_dict['loss']
            val_total_loss += loss.item()
    
    avg_val_loss = val_total_loss / len(dataloader_val)

    # Update the progress bar description with current epoch metrics
    progress_bar.set_description(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
    
    # Save model after each epoch
    torch.save(model.state_dict(), 'model_diffusion.pth')


Epoch 1000/1000, Loss: 0.7298, Validation Loss: 0.9340: 100%|██████████| 1000/1000 [23:13<00:00,  1.39s/it]


## Audio Generation

After training, we use the trained model to generate new audio samples. We perform ODE-based sampling starting from pure noise and integrating over time to produce latent representations, which are then decoded back into audio using the `EncoderDecoder`.

In [24]:
# Generate samples
num_samples = 5
num_steps = 100  # Number of diffusion steps
os.makedirs('generated_audio', exist_ok=True)
with torch.no_grad():
    for i in range(num_samples):
        print(f"Generating sample {i+1}/{num_samples}")
        # Get latents_shape from dataset's mean (assuming sequence length of 32)
        latents_shape = (1, dataset.mean.shape[0], 32)  # (batch_size=1, channels, seq_len)
        generated_latents = inference(rectified_flows, model, latents_shape, num_steps)
        # Unnormalize the generated latents
        mean_device = dataset.mean.to(generated_latents.device)
        std_device = dataset.std.to(generated_latents.device)
        generated_latents = generated_latents.squeeze(0)  # Remove batch dimension
        generated_latents = (generated_latents.permute(1, 0) * std_device + mean_device).permute(1, 0)
        # Decode the generated latents into audio
        generated_latents_np = generated_latents.cpu().numpy()  # Shape: [channels, seq_len]
        wv_rec = encdec.decode(generated_latents_np)
        # Save the audio file
        output_filename = f'generated_audio/diffusion_sample_{i + 1}.wav'
        sf.write(output_filename,
                 wv_rec[0],
                 samplerate=44100)

Generating sample 1/5


Generating sample 2/5


Generating sample 3/5


Generating sample 4/5

Generating sample 5/5


## Play Generated Audio

Finally, we can load and play the generated audio samples to listen to the results of our diffusion model.

In [25]:
# Use IPython audio player to play generated audio samples.
for i in range(1, num_samples + 1):
    output_filename = f'generated_audio/diffusion_sample_{i}.wav'
    print(f"Playing {output_filename}")
    ipd.display(ipd.Audio(output_filename))

Playing generated_audio/diffusion_sample_1.wav


Playing generated_audio/diffusion_sample_2.wav


Playing generated_audio/diffusion_sample_3.wav


Playing generated_audio/diffusion_sample_4.wav


Playing generated_audio/diffusion_sample_5.wav
